In [54]:
pip install pycountry pandas regex requests

Note: you may need to restart the kernel to use updated packages.


In [132]:
import requests
import zipfile
import os
import pycountry
import pandas as pd
import regex as re
from datetime import datetime

#The "download the temperature data" function
def DownloadFile(url, writeLocation):
    if requests.get(url).status_code == 200:
        with open(writeLocation, 'wb') as file:
            file.write(requests.get(url).content)
        return file
    else:
        return f"exited with {requests.get(url).status_code} as error"

def ExtractZip(zipFile, extractPath):
    try:
        with zipfile.ZipFile(zipFile, 'r') as zip_ref:
            zip_ref.extractall(extractPath)
        return extractPath
    except Exception as e:
        return f"Error: {e}"

def DownloadExtract(url, path):
    ExtractZip(DownloadFile(url, path), path)
    

def GenerateTemperatureMapping(folderPath):
    list = []
    STAIDCountry = {}
    reg = r"[\s]*(\d+),([\w\W\s]*),([a-zA-Z]{2})"
    EUList = ['BE', 'BG', 'CZ', 'DK', 'DE', 'EE', 'IE', 'EL', 'ES', 'FR', 'HR', 'IT', 'CY', 'LV', 'LT', 'LU', 'HU', 'MT', 'NL', 'AT', 'PL', 'PT', 'RO', 'SI', 'SK', 'FI', 'SE', 'GR']
    tempList = []
    dateList = []
    start = False
    
    #Lees het Satations.txt bestand,dit heeft de index van .txt bestanden->landen
    with open(folderPath + 'stations.txt') as f:
        lines = f.readlines()
        lines = lines[17:]
        f.close()
    
    #Genereer de dictionary die ID : land bijhoudt
    #TODO: Dictionary [land] : [lijst aan IDs] als STAIDCountry?
    for line in lines:
        m = re.match(reg, line)
        if m:
            if m.group(3) not in list:
                list.append(m.group(3))
            country = pycountry.countries.get(alpha_2=m.group(3)).name
            id = str(m.group(1)).zfill(5)
            STAIDCountry[id] = country
            
    # Voor elk .txt bestand, match de ID (gelezen door regex) met de key in de dictionary, en hernoem het bestand, zo genereert het LAND[ID].txt files 
    for file in os.listdir(folderPath):
        reg = r"TG_STAID0(\d*)"
        match = re.match(reg, file)
        if not match:
            continue
        id = match.group(1)
        newName = STAIDCountry[id] + id + '.txt'
        os.rename(os.path.join(folderPath, file), os.path.join(folderPath, newName))

    
    for txtFile in os.listdir(folderPath):
        tempList = []
        dateList = []
        start = False
        temp = []
        
        with open(folderPath + txtFile) as f:
            lines = f.readlines()
            f.close()
            
        for line in lines:
            reg = r"[\d]+,\s*\d+,(\d{8}),\s*([-\d]+),\s+(\d)"
            m = re.match(reg, line.strip())
            if m:
                if m.group(3) != "0" and m.group(3) != "1":
                    continue   
                # /10 omdat alle temperatuure in stappen van 0.1 graden celcius
                dateList.append(datetime.strptime(m.group(1), "%Y%m%d").date())
                tempList.append(int(m.group(2))/10)
            else:
                temp.append(line)
                
        df = pd.DataFrame({'Date': dateList, 'Temperature' : tempList})
        if df.empty:
            print(df.head(), "with text file:", txtFile)
            print(txtFile)
# temperatuur is in 0.1C, dus alles x 0.1 voordat het de database in geflikkerd wordt


GenerateTemperatureMapping('data/temp/')

Empty DataFrame
Columns: [Date, Temperature]
Index: [] with text file: stations.txt
stations.txt


KeyboardInterrupt: 

In [99]:
import regex as re
import pandas as pd

def GenerateTemperatureMapping(filePath):
    tempList = []
    dateList = []
    start = False

    with open(filePath) as f:
        lines = f.readlines()
        f.close()
    return lines
    for line in GenerateTemperatureMapping("data/Austria00016.txt"):
        reg = r"[\d]+,\d+,(\d{8}),\s*([-\d]+),\s+(\d)"
        m = re.match(reg, line.strip())
        if m:
            if m.group(3) != "0":
                continue   
            # /10 omdat alle temperatuure in stappen van 0.1 graden celcius
            dateList.append(datetime.strptime(m.group(1), "%Y%m%d").date())
            tempList.append(int(m.group(2))/10)
            
    df = pd.DataFrame({'Date': dateList, 'Temperature' : tempList})
    print(df.head())



         Date  Temperature
0  1855-02-01         -6.3
1  1855-02-02         -8.0
2  1855-02-03        -12.9
3  1855-02-04         -8.9
4  1855-02-05         -4.4
